<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MCP_DEMO_CONSEJO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/spaces/burtenshaw/karpathy-llm-council

In [ ]:
from gradio_client import Client

# Initialize the client
client = Client("burtenshaw/karpathy-llm-council")

from google.colab import userdata
openroute=userdata.get('OPENROUTER_API_KEY')

In [ ]:
!pip install autogen -q

In [21]:
from openai import OpenAI
from google.colab import userdata

OPENROUTER_API_KEY=userdata.get('OPENROUTER_API_KEY')
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY is not set")

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="qwen/qwen3-max",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of Agentic AI?"
    }
  ]
)
print(completion.choices[0].message.content)

**Agentic AI** refers to artificial intelligence systems that can act autonomously, make decisions, and take actions to achieve specific goals—often with a degree of independence, adaptability, and intentionality. Unlike traditional AI models that respond passively to user inputs (e.g


In [22]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

from google.colab import userdata

api_key=userdata.get("DEEPSEEK_API_KEY")

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

response_reasoner = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that can solve logic puzzles."},
        {"role": "user", "content": "There are three boxes. One contains only apples, one contains only oranges, and one contains both apples and oranges. Each box is incorrectly labeled. You can only open one box and take out one fruit. By looking at the fruit, how can you label all the boxes correctly?"},
    ],
    stream=False
)

print(response_reasoner.choices[0].message.content)

To correctly label all boxes, open the box labeled **"Apples and Oranges"** and take out one fruit. Since all boxes are incorrectly labeled, this box cannot contain both fruits. Therefore:

- If you take out an **apple**, this box contains only apples.  
  Then:  
  - The box labeled **"Oranges"** cannot contain only oranges (incorrect label) and cannot contain only apples (already identified), so it must contain both apples and oranges.  
  - The box labeled **"Apples"** must then contain only oranges (the only remaining option).

- If you take out an **orange**, this box contains only oranges.  
  Then:  
  - The box labeled **"Apples"** cannot contain only apples (incorrect label) and cannot contain only oranges (already identified), so it must contain both apples and oranges.  
  - The box labeled **"Oranges"** must then contain only apples.

This method allows you to deduce the correct contents of all three boxes after opening only one box and examining one fruit.


In [24]:
import os
import autogen
from google.colab import userdata

# --- 1. CONFIGURE API KEYS AND MODELS ---

# Fetch all required API Keys from Colab secrets
try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY not found in Colab secrets. Please set it.")
    # WARNING: Added a check for DeepSeek key, as it's required for the DeepSeek agent
    if not DEEPSEEK_API_KEY:
        print("WARNING: DEEPSEEK_API_KEY not found. DeepSeek Reasoner agent will likely fail.")

    os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

except ValueError as e:
    print(f"Error: {e}")
    raise SystemExit(e)

# Define Base URLs and API Types
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
DEEPSEEK_BASE_URL = "https://api.deepseek.com"
API_TYPE = "openai"

# Expanded list of Council Member models (5 Agents)
council_models = [
    # Council Member 1 (OpenRouter)
    {
        "model": "meta-llama/llama-3.1-8b-instruct",
        "api_key": OPENROUTER_API_KEY,
        "base_url": OPENROUTER_BASE_URL,
        "api_type": API_TYPE
    },
    # Council Member 2 (OpenRouter)
    {
        "model": "mistralai/mistral-7b-instruct",
        "api_key": OPENROUTER_API_KEY,
        "base_url": OPENROUTER_BASE_URL,
        "api_type": API_TYPE
    },
    # Council Member 3 (OpenRouter) - FIXED: Using stable Mixtral 8x7B
    {
        "model": "mistralai/mixtral-8x7b-instruct",
        "api_key": OPENROUTER_API_KEY,
        "base_url": OPENROUTER_BASE_URL,
        "api_type": API_TYPE
    },
    # NEW Member 4: Qwen3-Max (OpenRouter)
    {
        "model": "qwen/qwen3-max",
        "api_key": OPENROUTER_API_KEY,
        "base_url": OPENROUTER_BASE_URL,
        "api_type": API_TYPE
    },
    # NEW Member 5: DeepSeek Reasoner (DeepSeek API)
    {
        "model": "deepseek-reasoner",
        "api_key": DEEPSEEK_API_KEY,
        "base_url": DEEPSEEK_BASE_URL,
        "api_type": API_TYPE
    }
]

# The Chairman Model (via OpenRouter)
chairman_model_config = {
    "model": "meta-llama/llama-3.1-70b-instruct",
    "api_key": OPENROUTER_API_KEY,
    "base_url": OPENROUTER_BASE_URL,
    "api_type": API_TYPE
}

# --- 2. DEFINE THE AGENTS (The Council Members and Chairman) ---

council_agents = []
for i, config in enumerate(council_models):
    model_name = config['model'].split('/')[-1]
    agent_name = f"Council_Member_{i+1}_{model_name}"

    council_agents.append(
        autogen.AssistantAgent(
            name=agent_name,
            llm_config={
                "config_list": [config],
            },
            system_message=f"You are {agent_name}, an expert LLM using the {config['model']} model. Provide a factual and comprehensive answer. Critically review other members' responses."
        )
    )

# The Chairman/User Proxy Agent (Performs synthesis)
chairman = autogen.UserProxyAgent(
    name="Council_Chairman",
    llm_config={
        "config_list": [chairman_model_config],
    },
    system_message=(
        "You are the Council Chairman. Review all opinions and conflicts from the members. "
        "Synthesize their best points and produce a single, final, reliable report. "
        "The report MUST end with the word 'report.' to signal termination."
    ),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=15,
    is_termination_msg=lambda x: x.get("content", "").strip().endswith("report."),
)

# --- 3. INITIATE THE COUNCIL DELIBERATION ---

# Define the GroupChat object (6 agents in total: 5 members + 1 chairman)
council_group = autogen.GroupChat(
    agents=[chairman] + council_agents,
    messages=[],
    max_round=20,
)

# Define the GroupChatManager
council_manager = autogen.GroupChatManager(
    groupchat=council_group,
    llm_config={
        "config_list": [chairman_model_config],
    },
)

# Start the conversation
question = "What are the key trade-offs between using a single powerful LLM versus a multi-agent LLM council architecture?"

print(f"--- Initiating Expanded OpenRouter/OpenSource Council for: '{question}' ---")

# The chairman initiates the chat
chat_result = chairman.initiate_chat(
    council_manager,
    message=question
)

# Extract and print the final synthesized response from the Chairman
final_answer = chat_result.summary

print("\n" + "="*50)
print("      *** LLM COUNCIL FINAL SYNTHESIS (Expanded Open Source Council) ***")
print("="*50)
print(final_answer)
print("="*50)

--- Initiating Expanded OpenRouter/OpenSource Council for: 'What are the key trade-offs between using a single powerful LLM versus a multi-agent LLM council architecture?' ---
Council_Chairman (to chat_manager):

What are the key trade-offs between using a single powerful LLM versus a multi-agent LLM council architecture?

--------------------------------------------------------------------------------
[autogen.oai.client: 12-04 09:13:56] {704} WARNING - Model meta-llama/llama-3.1-70b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



Next speaker: Council_Member_1_llama-3.1-8b-instruct

[autogen.oai.client: 12-04 09:14:29] {704} WARNING - Model meta-llama/llama-3.1-8b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Council_Member_1_llama-3.1-8b-instruct (to chat_manager):

As a LLM expert, I'll provide a comprehensive review of the key trade-offs between using a single powerful LLM versus a multi-agent LLM council architecture.

**Single Powerful LLM:**

1.  **Scalability**: Single powerful LLMs can handle large amounts of data and scale up to a significant number of conversations simultaneously.
2.  **Efficiency**: A single LLM can process information more efficiently as it doesn't require communication between different agents.
3.  **Cost**: Maintaining a single powerful LLM is generally more cost-effective.
4.  **Complexity**: Managing a single LLM is less complex than managing multiple agents and their interactions.

**Multi-Agent LLM Council Architecture:**

1.  **Redundancy and Robustness**: A multi-agent council can provide redundancy and improve overall robustness, as multiple agents can work together to achieve a task.
2.  **Specialization**: Different agents can specialize in specific t


Next speaker: Council_Member_2_mistral-7b-instruct

[autogen.oai.client: 12-04 09:14:30] {704} WARNING - Model mistralai/mistral-7b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Council_Member_2_mistral-7b-instruct (to chat_manager):

 

--------------------------------------------------------------------------------
[autogen.oai.client: 12-04 09:14:33] {704} WARNING - Model meta-llama/llama-3.1-70b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


...

Council_Chairman (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 12-04 09:21:10] {704} WARNING - Model meta-llama/llama-3.1-70b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



Next speaker: Council_Chairman

[autogen.oai.client: 12-04 09:21:15] {704} WARNING - Model meta-llama/llama-3.1-70b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Council_Chairman (to chat_manager):

**FINAL REPORT**

**Council Chairman's Summary and Synthesis**

After careful review and analysis of the opinions and conflicts presented by Council Members 1 through 5, I am pleased to provide a comprehensive final report that synthesizes the best points and presents a reliable conclusion.

**Key Findings:**

1.  **Single Powerful LLM**: Offers advantages in scalability, efficiency, cost, and complexity. Suitable for tasks requiring holistic reasoning, low-latency responses, and predictable maintenance.
2.  **Multi-Agent LLM Council Architecture**: Provides benefits in redundancy, specialization, diversity of perspectives, and flexibility. Ideal for tasks requiring decomposable subproblems, high-stakes decision-making, and creative synthesis.

**Critical Insights:**

1.  **Emergent Coordination Complexity**: Multi-agent systems introduce non-trivial coordination overhead, which can lead to communication latency, debugging challenges, and containmen


Next speaker: Council_Chairman

[autogen.oai.client: 12-04 09:21:18] {704} WARNING - Model meta-llama/llama-3.1-70b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Council_Chairman (to chat_manager):

**

**End of Report.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (8fd01256-b08b-4ea4-acfc-ad9489fd8e4d): Maximum rounds (20) reached

      *** LLM COUNCIL FINAL SYNTHESIS (Expanded Open Source Council) ***
**

**End of Report.
